In [1]:
import pandas as pd

# Carga de los datset iniciales
print("Cargando datos de las fuentes csv...")
try:
    df_a = pd.read_csv("DatasetNombres/healthcare_dataset.csv")
    df_b = pd.read_csv("DatasetValores/synthetic_clinical_dataset.csv")
    print("Datos cargados correctamente")
    
except FileNotFoundError as e:
    print("Error: No se encontró uno de los archivos CSV especificados.")
    print(f"Detalles del error: {type(e).__name__} - {e}")
    
except Exception as e:
    print("Error inesperado durante la carga de datos.")
    print(f"Detalles del error: {type(e).__name__} - {e}")

Cargando datos de las fuentes csv...
Datos cargados correctamente


In [2]:
# visualización de información general de los dataset cargados
print("\n" + "=" * 80)
print("Información del primer dataset - healthcare.csv")
df_a.info()
print("Primeras 10 filas del dataset")
print(df_a.head(10))
print("Valores nulos de cada columna")
print(df_a.isnull().sum())


print("\n" + "=" * 80)
print("Información del segundo dataset - synthetic_clinical.csv")
df_b.info()
print("Primeras 10 filas del dataset")
print(df_b.head(10))
print("Valores nulos de cada columna")
print(df_b.isnull().sum())


Información del primer dataset - healthcare.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55500 entries, 0 to 55499
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                55500 non-null  object 
 1   Age                 55500 non-null  int64  
 2   Gender              55500 non-null  object 
 3   Blood Type          55500 non-null  object 
 4   Medical Condition   55500 non-null  object 
 5   Date of Admission   55500 non-null  object 
 6   Doctor              55500 non-null  object 
 7   Hospital            55500 non-null  object 
 8   Insurance Provider  55500 non-null  object 
 9   Billing Amount      55500 non-null  float64
 10  Room Number         55500 non-null  int64  
 11  Admission Type      55500 non-null  object 
 12  Discharge Date      55500 non-null  object 
 13  Medication          55500 non-null  object 
 14  Test Results        55500 non-null  object 
dtypes: f

In [3]:
print("Valores únicos a emparejar del dataset A:")
print(df_a['Gender'].unique())

print("Valores únicos a emparejar del dataset B:")
print(df_b['sex'].unique())
# normalización de la columna por la cual se van a integrar ambos dataset
df_a["Gender"] = df_a["Gender"].str.strip().str.lower()
df_b["sex"] = df_b["sex"].str.strip().str.lower()

# str.strip para eliminar espacios innecesarios
# str.lower para normalizar la escritura a un solo tipo

Valores únicos a emparejar del dataset A:
['Male' 'Female']
Valores únicos a emparejar del dataset B:
['Male' 'Female' 'Other']


In [4]:
# Separación del dataset a por genero - clasificacion de registros
df_a_male = df_a[df_a["Gender"] == "male"].reset_index(drop=True)
df_a_female = df_a[df_a["Gender"] == "female"].reset_index(drop=True)

'''
Se separa en dos conjuntos para ir tomando registros respectivamente según se empareje con el genero del registro x en el segundo dataset
El segundo dataset se va a recorrer de manera secuencial emparejando por genero los valores del primer dataset
.reset_index() Para crear un nuevo indice secuencial en los subconjuntos
drop = True para que no se agregue el indice del dataframe original como una columna
'''
print("\nValores en cada subconjunto")
print("Male:", df_a_male.shape)
print("Female:", df_a_female.shape)


Valores en cada subconjunto
Male: (27774, 15)
Female: (27726, 15)


In [5]:
# inicializamos una lista para cargar los registros combinados que se guardaran en el dataframe final
combined_rows = []

# tambien se inicializa los contadores para ir recorriendo las posiciones de los subconjuntos de generos
i_male = 0
i_female = 0

In [6]:
# como un valor de genero es other, vamos a generar data controlada para luego aplicar tecnicas de limpieza sober ciertas columnas
import random
aleatorios = 0


# recorremos todas las filas del segundo dataset
for _, row_b in df_b.iterrows():
    genero = row_b["sex"]

    if genero == "male":
        if i_male < len(df_a_male):
            row_a = df_a_male.iloc[i_male]
            i_male += 1
        else:
            # Si no quedan más hombres disponibles se añade nulo
            print("Se acabaron los registros de hombres")
            row_a = pd.Series()
    elif genero == "female":
        if i_female < len(df_a_female):
            row_a = df_a_female.iloc[i_female]
            i_female += 1
        else:
            # Si no quedan más mujeres disponibles va nulo
            print("Se acabaron los registros de mujeres")
            row_a = pd.Series()
    else:
        # Si existe un genero desconocido o distinto
        print(f"Genero desconocido #{aleatorios+1}: {genero}")
        if aleatorios < 100:
            # Escogemos aleatoriamente entre los subconjuntos de genero
            if random.choice(["male", "female"]) == "male" and len(df_a_male) > 0:
                random_index = random.randint(0, len(df_a_male) - 1)
                row_a_completa = df_a_male.iloc[random_index]
            elif len(df_a_female) > 0:
                random_index = random.randint(0, len(df_a_female) - 1)
                row_a_completa = df_a_female.iloc[random_index]
            else:
                row_a_completa = pd.Series()

            # Solo conversamos la columna del nombre
            if not row_a_completa.empty:
                row_a = pd.Series({col: row_a_completa[col] if col == "Name" else pd.NA for col in df_a.columns})
            else:
                row_a = pd.Series()
                
            aleatorios += 1
        else:
            # Si ya se hicieron 100 asignaciones, dejamos vacío
            row_a = pd.Series()
    
    # Combinamos las columnas de B con las de A
    combined_rows.append(pd.concat([row_b, row_a]))

print(len(combined_rows))
'''
iterrows() permite iterar fila por fila un df devolviendo index, Series - como no necesitamos el indice en la primera parte del for usamos _
en row_b["etiqeuta"] extraemos el campo especifico de esa fila
Clasificamos con el valor del genero en la fila x del segundo dataset y depende el caso tomamos uno de los registros del subconjunto
siempre y cuando aún haya registros por tomar
Con iloc hacemos acceso posicional obteniendo la fila i del subconjunto y avanzamos hacia el siguiente registro del subconjunto del primer df
pd.Series() añade una fila vacia
.concat() concatena verticalmente los dos Series (por índice/etiqueta), resultando en un único Series
.append() añadimos esa serie a la lista de resultados
'''

Genero desconocido #1: other
Genero desconocido #2: other
Genero desconocido #3: other
Genero desconocido #4: other
Genero desconocido #5: other
Genero desconocido #6: other
Genero desconocido #7: other
Genero desconocido #8: other
Genero desconocido #9: other
Genero desconocido #10: other
Genero desconocido #11: other
Genero desconocido #12: other
Genero desconocido #13: other
Genero desconocido #14: other
Genero desconocido #15: other
Genero desconocido #16: other
Genero desconocido #17: other
Genero desconocido #18: other
Genero desconocido #19: other
Genero desconocido #20: other
Genero desconocido #21: other
Genero desconocido #22: other
Genero desconocido #23: other
Genero desconocido #24: other
Genero desconocido #25: other
Genero desconocido #26: other
Genero desconocido #27: other
Genero desconocido #28: other
Genero desconocido #29: other
Genero desconocido #30: other
Genero desconocido #31: other
Genero desconocido #32: other
Genero desconocido #33: other
Genero desconocido 

'\niterrows() permite iterar fila por fila un df devolviendo index, Series - como no necesitamos el indice en la primera parte del for usamos _\nen row_b["etiqeuta"] extraemos el campo especifico de esa fila\nClasificamos con el valor del genero en la fila x del segundo dataset y depende el caso tomamos uno de los registros del subconjunto\nsiempre y cuando aún haya registros por tomar\nCon iloc hacemos acceso posicional obteniendo la fila i del subconjunto y avanzamos hacia el siguiente registro del subconjunto del primer df\npd.Series() añade una fila vacia\n.concat() concatena verticalmente los dos Series (por índice/etiqueta), resultando en un único Series\n.append() añadimos esa serie a la lista de resultados\n'

In [7]:
# Finalmente creamos el nuevo dataframe en base a la lista de series que se obtiene como resultado de la integración
df_data_final = pd.DataFrame(combined_rows)
print("\n" + "=" * 100)
print("Datos finales del dataset")
df_data_final.info()

print("\n" + "=" * 100)
print("Datos nulos del dataset")
print(df_data_final.isnull().sum())


Datos finales del dataset
<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 0 to Unnamed 9995
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   patient_id          10000 non-null  int64  
 1   age                 10000 non-null  int64  
 2   sex                 10000 non-null  object 
 3   bmi                 10000 non-null  float64
 4   systolic_bp         10000 non-null  float64
 5   diastolic_bp        10000 non-null  float64
 6   glucose             10000 non-null  float64
 7   cholesterol         10000 non-null  float64
 8   creatinine          10000 non-null  float64
 9   diabetes            10000 non-null  int64  
 10  hypertension        10000 non-null  int64  
 11  diagnosis           10000 non-null  object 
 12  readmission_30d     10000 non-null  int64  
 13  mortality           10000 non-null  int64  
 14  Name                9984 non-null   object 
 15  Age                 9884

In [8]:
# Filtramos solo los datos necesarios que van a ser usados
columnas_ordenadas = [
    "patient_id", "Name", "age", "sex", "Blood Type",
    "bmi", "systolic_bp", "diastolic_bp", "glucose",
    "cholesterol", "creatinine", "diabetes", "hypertension",
    "diagnosis", "readmission_30d", "mortality", "Medical Condition"
]

columnas_finales = [col for col in columnas_ordenadas if col in df_data_final.columns]

df_data_final = df_data_final[columnas_finales]

df_data_final.info()
print("="*80)
print(df_data_final.head(10))

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 0 to Unnamed 9995
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patient_id         10000 non-null  int64  
 1   Name               9984 non-null   object 
 2   age                10000 non-null  int64  
 3   sex                10000 non-null  object 
 4   Blood Type         9884 non-null   object 
 5   bmi                10000 non-null  float64
 6   systolic_bp        10000 non-null  float64
 7   diastolic_bp       10000 non-null  float64
 8   glucose            10000 non-null  float64
 9   cholesterol        10000 non-null  float64
 10  creatinine         10000 non-null  float64
 11  diabetes           10000 non-null  int64  
 12  hypertension       10000 non-null  int64  
 13  diagnosis          10000 non-null  object 
 14  readmission_30d    10000 non-null  int64  
 15  mortality          10000 non-null  int64  
 16  Medical Condition  9

In [9]:
print("Valores nulos en la columna Name: ", df_data_final["Name"].isna().sum())
print("Registros en el dataset: ", len(df_data_final))

Valores nulos en la columna Name:  16
Registros en el dataset:  10000


In [10]:
# limpiamos la columna Name - eliminando los registros con nombres nulos
df_data_final = df_data_final.dropna(subset=["Name"])
print("Valores nulos en la columna Name: ", df_data_final["Name"].isna().sum())
print("Registros en el dataset", len(df_data_final))

Valores nulos en la columna Name:  0
Registros en el dataset 9984


In [11]:
# Imputamos los nulos de blood type con la moda
if "Blood Type" in df_data_final.columns:
    # Calculamos la moda
    moda_blood = df_data_final["Blood Type"].mode()[0]
    print(f"Moda de la columna 'Blood Type': {moda_blood}")

    # Rellenamos valores nulos con la moda
    # df_data_final["Blood Type"].fillna(moda_blood, inplace=True)
    df_data_final.fillna({'Blood Type': moda_blood}, inplace=True)

print("Valores nulos en Blood Type:", df_data_final['Blood Type'].isnull().sum())

# Imputamos los nulos de Medical condition igual con la moda
if 'Medical Condition' in df_data_final.columns:
    moda_medical_condition = df_data_final["Medical Condition"].mode()[0]
    print(f"Moda de la columna 'Medical Condition': {moda_medical_condition}")

    # df_data_final["Medical Condition"].fillna(moda_medical_condition, inplace=True)
    df_data_final["Medical Condition"] = df_data_final["Medical Condition"].fillna(moda_medical_condition)

print("Valores nulos en Medical Condition:", df_data_final['Medical Condition'].isnull().sum())

Moda de la columna 'Blood Type': A+
Valores nulos en Blood Type: 0
Moda de la columna 'Medical Condition': Arthritis
Valores nulos en Medical Condition: 0


In [12]:
# Formateamos la columna Name y la columna sex
if "Name" in df_data_final.columns:
    df_data_final["Name"] = (
        df_data_final["Name"]
        .astype(str)
        .str.strip()          # elimina espacios extra
        .str.lower()          # todo en minúscula
        .str.title()          # primera letra mayúscula
    )

if "sex" in df_data_final.columns:
    df_data_final["sex"] = (
        df_data_final["sex"]
        .astype(str)
        .str.strip()
        .str.title()
    )

In [13]:
# Revisamos finalmente como quedo el datafram
print("============= DATAFRAME FINAL ==============")
df_data_final.info()

print("\n" + "=" * 100)
print("Primero 5 registros")
print(df_data_final.head())

print("\n" + "=" * 100)
print("Analisis estadistico general")
print(df_data_final.describe())

============= DATAFRAME FINAL ==============
<class 'pandas.core.frame.DataFrame'>
Index: 9984 entries, 0 to Unnamed 9995
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patient_id         9984 non-null   int64  
 1   Name               9984 non-null   object 
 2   age                9984 non-null   int64  
 3   sex                9984 non-null   object 
 4   Blood Type         9984 non-null   object 
 5   bmi                9984 non-null   float64
 6   systolic_bp        9984 non-null   float64
 7   diastolic_bp       9984 non-null   float64
 8   glucose            9984 non-null   float64
 9   cholesterol        9984 non-null   float64
 10  creatinine         9984 non-null   float64
 11  diabetes           9984 non-null   int64  
 12  hypertension       9984 non-null   int64  
 13  diagnosis          9984 non-null   object 
 14  readmission_30d    9984 non-null   int64  
 15  mortality          9984 

In [14]:
# Exportamos nuestro dataset de resultado como un archivo csv
df_data_final.to_csv("dataset_final.csv", index=False)
print("Dataset combinado con éxito...")

Dataset combinado con éxito...
